In [2]:
#import necessary libraries
import asyncio
import aiohttp
import pycurl
import json
from io import BytesIO

In [3]:
def perform_ntlm_authenticated_request(url, username, password):
    buffer = BytesIO()
    curl = pycurl.Curl()
    curl.setopt(pycurl.URL, url)
    curl.setopt(pycurl.USERPWD, f"{username}:{password}")
    curl.setopt(pycurl.HTTPAUTH, pycurl.HTTPAUTH_NTLM)
    curl.setopt(pycurl.WRITEFUNCTION, buffer.write)
    
    # Disable TLS verification
    curl.setopt(pycurl.SSL_VERIFYPEER, False)
    curl.setopt(pycurl.SSL_VERIFYHOST, False)
    
    curl.perform()
    curl.close()
    body = buffer.getvalue().decode('utf-8')
    buffer.close()
    return body

def findFolders(listFolder,out):
    #out = []
    #print("def was call for listFolder ID = " + str(listFolder))
    out.append(listFolder)
    username = "administrator"
    password = "77@NguyenQuyDuc"
    #for i in range(len(listFolder)):
    url = "https://192.168.10.202/pivision/utility/api/v1/folders?folderid="+str(listFolder)
    childfolder_json = json.loads(perform_ntlm_authenticated_request(url,username,password))
    for Id in range(len(childfolder_json["Items"])):
        #print("loop in "+str(Id))
        #out.append(childfolder_json["Items"][Id]["Id"])
        #print(childfolder_json["Items"][Id]["Id"])
        if childfolder_json["Items"][Id]["HasChildren"] == True:
            findFolders(childfolder_json["Items"][Id]["Id"],out)
    #print("done loop in")
    return None

def findDislays(folderId,display_id):
    username = "administrator"
    password = "77@NguyenQuyDuc"
    for Id in folderId:
        url = f"https://192.168.10.202/pivision/utility/api/v1/displays?folderid={Id}"
        display_json = json.loads(perform_ntlm_authenticated_request(url,username,password))
        for k in range(len(display_json["Items"])):
            display_id.append(display_json["Items"][k]["Id"])
    return None

In [4]:
async def fetch_data(url,username,password,display_dict):        
    temp_source = []   
    display_json = json.loads(perform_ntlm_authenticated_request(url,username,password))
    for symbol in range(len(display_json["Display"]["Symbols"])):
        if "DataSources" in display_json["Display"]["Symbols"][symbol]:
            #print(display_json["Display"]["Symbols"][symbol]["DataSources"])
            temp_source.append(display_json["Display"]["Symbols"][symbol]["DataSources"])
        display_dict[display_json["Display"]["Name"]] = temp_source
    return None
        
async def fetch_all_data(urls,username,password,display_dict):
    tasks = [fetch_data(url,username,password,display_dict) for url in urls]
    return await asyncio.gather(*tasks)

In [8]:
import time
start_time = time.time()

listFolder = ""
out=[]
username = "administrator"
password = "77@NguyenQuyDuc"
findFolders(listFolder,out) #not 'out' list has been filled
#print(out)

display_id = [] 
allFolders = out
findDislays(allFolders,display_id)

#this is the total number of displays in PI Vision
#print(len(display_id)) 

display_dict = {}
urls = [f"https://192.168.10.202/pivision/utility/api/v1/displays/{graphic}/export" for graphic in display_id]
await fetch_all_data(urls,username,password,display_dict)

inverse = {}
for k, v in display_dict.items():
    if v != []:
        for x in v:
            if isinstance(x, list):
                for item in x:
                    inverse.setdefault(item, []).append(k)
            else:
                inverse.setdefault(x, []).append(k)
                
print("The dictionary has " + str(len(inverse)) + " entries.")
print("Exec time lasted --- %s seconds ---" % (time.time() - start_time))

The dictionary has 1900 entries.
--- 252.1023554801941 seconds ---


In [ ]:
print(inverse)